In [1]:
# from openai import AzureOpenAI
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv()

# Azure OpenAI 클라이언트 초기화
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    azure_endpoint=os.getenv("SKCC_AZURE_ENDPOINT"),
    api_key=os.getenv("SKCC_AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview",
    temperature=0.7
)

# 채팅 호출 예시
response = llm.invoke("안녕?")

print(response.content)

안녕하세요! 😊 어떻게 도와드릴까요?


In [16]:
from typing import List, Dict, Any, Optional
from dataclasses import dataclass
from langchain.tools import tool
import re
#from typing import Literal
#from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage
#from langchain.agents import create_agent
from langgraph.prebuilt import create_react_agent
from typing import Literal
#from langgraph.types import Command
from langchain_core.messages import HumanMessage
from pydantic import BaseModel
import json
import os
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

with open('snowflake_global_weather_climate_data_for_bi_table_schema', 'r') as f:

    json_data = json.load(f)

print(json.dumps(json_data) )

global full_schema_json

full_schema_json = json_data

@dataclass
class TableSchema:
    table_name: str
    columns: List[str]
    descriptions: Dict[str, str] = None
    samples: List[Dict[str, Any]] = None

def _tokenize(s: str) -> List[str]:
    return re.findall(r"[A-Za-z0-9_]+", s.lower())

def _score(text: str, q_tokens: List[str]) -> int:
    tks = _tokenize(text)
    return sum(1 for qt in q_tokens if qt in tks)

def _table_text_blob(ts: TableSchema) -> str:
    col_part = " ".join(ts.columns)
    desc_part = " ".join([(ts.descriptions or {}).get(c, "") for c in ts.columns])
    return f"{ts.table_name} {col_part} {desc_part}"

class FindTablesInput(BaseModel):
    query: str
    top_k: int = 5

class FindTablesOutput(BaseModel):
    table_name: str
    score: float

@tool("find_relevant_tables", args_schema=FindTablesInput, return_direct=False)
def find_relevant_tables(query: str,  top_k: int = 5) -> List[FindTablesOutput]:
    """
    사용자 질문(query)과 DB 스키마(db_schema)를 입력받아 관련도가 높은 테이블 top_k개를 반환
    반환 형식: [{"table_name": "...", "score": 7}, ...]
    """
    # db_schema가 주어지지 않으면 미리 로드한 full_schema_json 사용
    schema_data = full_schema_json
    q_tokens = _tokenize(query)
    scored = []
    for t in schema_data:
        ts = TableSchema(
            table_name=t["table_name"],
            columns=t.get("column_names", []),
            descriptions={c: d or "" for c, d in zip(t.get("column_names", []), t.get("description", []) or [])},
            samples=t.get("sample_rows", [])
        )
        sc = _score(_table_text_blob(ts), q_tokens)
        scored.append({"table_name": ts.table_name, "score": sc})
    scored.sort(key=lambda x: x["score"], reverse=True)
    return scored[:top_k]

# ⚠️ get_table_columns 도구에도 Optional db_schema 적용
class GetColumnsInput(BaseModel):
    table_name: str

@tool("get_table_columns", args_schema=GetColumnsInput, return_direct=False)
def get_table_columns(table_name: str) -> Dict[str, Any]:
    """
    단일 테이블의 컬럼, 타입, 설명을 반환.
    """
    schema_data = full_schema_json
    for t in schema_data:
        if t["table_name"].lower() == table_name.lower():
            return {
                "table_name": t["table_name"],
                "columns": t.get("column_names", []),
                "types": t.get("column_types", []),
                "descriptions": t.get("description", []),
                "samples": t.get("sample_rows", [])
            }
    return {"table_name": table_name, "error": "not found"}


llm_selector = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    azure_endpoint=os.getenv("SKCC_AZURE_ENDPOINT"),
    api_key=os.getenv("SKCC_AZURE_OPENAI_KEY"),
    api_version="2024-02-15-preview",
    temperature=0.7
)


selector_tools = [find_relevant_tables, get_table_columns]


SELECTOR_SYSTEM_PROMPT = """
당신은 Oracle Text-to-SQL용 MAC-SQL Selector Agent입니다.
당신의 임무는 **사용자 질문과 전체 DB 스키마(JSON 리스트)**를 기반으로 관련된 테이블과 컬럼만 선택하는 것입니다.

제공된 도구를 사용하여 다음을 수행합니다:
1) 가장 관련성 높은 후보 테이블을 찾습니다.
2) 해당 테이블의 컬럼과 설명을 가져옵니다.
그 후, 아래와 같은 엄격한 JSON 형식으로만 결과를 반환해야 합니다:
{
  "selected_tables": ["TBL1", "TBL2", ...],
  "selected_columns": {
    "TBL1": ["col_a", "col_b"],
    "TBL2": ["col_x"]
  },
  "rationale": "why these tables/columns are chosen in Korean",
  "confidence": 0.0_to_1.0
}

규칙:
- Oracle SQL 문법만을 기준으로 합니다.
- 정확성 우선: 많기보다는 적더라도 정확한 테이블/컬럼을 선택하세요.
- DB 스키마에 존재하지 않는 테이블이나 컬럼을 절대 만들어내지 마세요.
- 결과는 반드시 JSON 형식만 출력하며, 백틱(`)이나 설명 문장은 포함하지 마세요.
- 대답은 반드시 한글로만 해주세요.
"""

selector_agent = create_react_agent(
    llm_selector,
    tools=selector_tools,
    prompt=SELECTOR_SYSTEM_PROMPT,
)

#nl_question = "각 국가의 최신 통화코드를 알려줘"
#nl_question = "How did each salesperson's annual total sales compare to their annual sales quota? Provide the difference between their total sales and the quota for each year, organized by salesperson and year."
nl_question = """
오늘이 2024년 4월 1일이라고 가정할 때, 전년도 첫 2주가 지난 후 일주일 동안 미국 각 우편번호 지역에 6인치(15cm) 이상의 일일 강설량을 알고 싶습니다. 
우편번호, 날짜, 강설량을 표시해 주세요.
"""





result = selector_agent.invoke({
    "messages": [
        HumanMessage(content=f"query:\n{nl_question}")
    ]
})

[{"table_name": "CLIMATOLOGY_DAY", "table_fullname": "GLOBAL_WEATHER__CLIMATE_DATA_FOR_BI.STANDARD_TILE.CLIMATOLOGY_DAY", "column_names": ["AVG_OF__DAILY_MAX_TEMPERATURE_WETBULB_F", "AVG_OF__DAILY_AVG_TEMPERATURE_AIR_F", "AVG_OF__DAILY_AVG_PRESSURE_SURFACE_MB", "STD_OF__DAILY_MAX_HUMIDITY_RELATIVE_PCT", "STD_OF__POS_DAILY_TOT_SNOWFALL_IN", "POSTAL_CODE", "FRQ_OF__POS_DAILY_TOT_SNOWFALL_IN_PCT", "STD_OF__DAILY_MAX_WIND_SPEED_10M_MPH", "AVG_OF__DAILY_MIN_TEMPERATURE_AIR_F", "FRQ_OF__DAILY_TOT_PRECIPITATION_ZERO_IN_PCT", "STD_OF__POS_DAILY_SNOWDEPTH_IN", "AVG_OF__DAILY_MIN_CLOUD_COVER_TOT_PCT", "AVG_VEC_OF__DAILY_AVG_VEC_WIND_DIRECTION_10M_DEG", "AVG_OF__DAILY_AVG_CLOUD_COVER_TOT_PCT", "AVG_OF__POS_DAILY_TOT_PRECIPITATION_IN", "AVG_OF__DAILY_MAX_PRESSURE_SURFACE_MB", "AVG_OF__DAILY_MAX_RADIATION_SOLAR_TOTAL_WPM2", "STD_OF__DAILY_MAX_CLOUD_COVER_TOT_PCT", "PCT_FRQ_OF__DAILY_TOT_SNOWFALL_250_GTR_IN_PCT", "STD_OF__DAILY_MIN_PRESSURE_MEAN_SEA_LEVEL_MB", "AVG_OF__DAILY_AVG_RADIATION_SOLAR_TOTA

C:\Users\worb1\AppData\Local\Temp\ipykernel_17112\2985793254.py:138: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  selector_agent = create_react_agent(


In [17]:
result

{'messages': [HumanMessage(content='query:\n\n오늘이 2024년 4월 1일이라고 가정할 때, 전년도 첫 2주가 지난 후 일주일 동안 미국 각 우편번호 지역에 6인치(15cm) 이상의 일일 강설량을 알고 싶습니다. \n우편번호, 날짜, 강설량을 표시해 주세요.\n', additional_kwargs={}, response_metadata={}, id='4d2430bb-c92d-498c-b946-cf108c1d2339'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 490, 'total_tokens': 533, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_b54fe76834', 'id': 'chatcmpl-CVI3uJbF9Tqr6SXd51GlAKT4I4K1r', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'sa

In [18]:
import sqlite3
from pprint import pprint

# 1. DB 연결
conn = sqlite3.connect("Chinook_Sqlite.sqlite")
cursor = conn.cursor()

# 2. 자연어 질문
question = "가장 많이 판매된 앨범 이름은?"

# 3. (Text2SQL 결과라고 가정) SQL 변환 예시
sql = """
SELECT Album.Title, COUNT(InvoiceLine.InvoiceLineId) AS SalesCount
FROM InvoiceLine
JOIN Track ON InvoiceLine.TrackId = Track.TrackId
JOIN Album ON Track.AlbumId = Album.AlbumId
GROUP BY Album.AlbumId
ORDER BY SalesCount DESC
LIMIT 1;
"""

# 4. 쿼리 실행
cursor.execute(sql)
result = cursor.fetchall()

# 5. 출력
print(f"질문: {question}")
pprint(result)

conn.close()

질문: 가장 많이 판매된 앨범 이름은?
[('Minha Historia', 27)]
